In [84]:
# import dependencies
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import Get_GPUcost
import pymongo
import datetime
import pandas_datareader as pdr

### EXTRACT Data for Crypto

In [85]:
#Scrape using request-JSON

# Save config information.
url = "https://whattomine.com/coins.json?eth=true&factor%5Beth_hr%5D=90.0&factor%5Beth_p%5D=420.0&e4g=true&factor%5Be4g_hr%5D=90.0&factor%5Be4g_p%5D=420.0&zh=true&factor%5Bzh_hr%5D=63.0&factor%5Bzh_p%5D=360.0&cnh=true&factor%5Bcnh_hr%5D=2880.0&factor%5Bcnh_p%5D=330.0&cng=true&factor%5Bcng_hr%5D=2280.0&factor%5Bcng_p%5D=360.0&cnf=true&factor%5Bcnf_hr%5D=4950.0&factor%5Bcnf_p%5D=330.0&cx=true&factor%5Bcx_hr%5D=3.3&factor%5Bcx_p%5D=360.0&eqa=true&factor%5Beqa_hr%5D=285.0&factor%5Beqa_p%5D=360.0&cc=true&factor%5Bcc_hr%5D=7.8&factor%5Bcc_p%5D=390.0&cr29=true&factor%5Bcr29_hr%5D=7.2&factor%5Bcr29_p%5D=390.0&ct31=true&factor%5Bct31_hr%5D=1.8&factor%5Bct31_p%5D=360.0&ct32=true&factor%5Bct32_hr%5D=0.48&factor%5Bct32_p%5D=360.0&eqb=true&factor%5Beqb_hr%5D=46.5&factor%5Beqb_p%5D=420.0&rmx=true&factor%5Brmx_hr%5D=1410.0&factor%5Brmx_p%5D=270.0&ns=true&factor%5Bns_hr%5D=2460.0&factor%5Bns_p%5D=450.0&al=true&factor%5Bal_hr%5D=177.0&factor%5Bal_p%5D=390.0&ops=true&factor%5Bops_hr%5D=14.7&factor%5Bops_p%5D=360.0&eqz=true&factor%5Beqz_hr%5D=42.0&factor%5Beqz_p%5D=390.0&zlh=true&factor%5Bzlh_hr%5D=42.0&factor%5Bzlh_p%5D=360.0&kpw=true&factor%5Bkpw_hr%5D=39.0&factor%5Bkpw_p%5D=510.0&ppw=true&factor%5Bppw_hr%5D=28.2&factor%5Bppw_p%5D=420.0&x25x=true&factor%5Bx25x_hr%5D=2.49&factor%5Bx25x_p%5D=240.0&fpw=true&factor%5Bfpw_hr%5D=43.5&factor%5Bfpw_p%5D=510.0&vh=true&factor%5Bvh_hr%5D=1.32&factor%5Bvh_p%5D=360.0&factor%5Bcost%5D=0.1&factor%5Bcost_currency%5D=USD&sort=Profitability24&volume=0&revenue=24h&factor%5Bexchanges%5D%5B%5D=&factor%5Bexchanges%5D%5B%5D=binance&factor%5Bexchanges%5D%5B%5D=bitfinex&factor%5Bexchanges%5D%5B%5D=bitforex&factor%5Bexchanges%5D%5B%5D=bittrex&factor%5Bexchanges%5D%5B%5D=coinex&factor%5Bexchanges%5D%5B%5D=dove&factor%5Bexchanges%5D%5B%5D=exmo&factor%5Bexchanges%5D%5B%5D=gate&factor%5Bexchanges%5D%5B%5D=graviex&factor%5Bexchanges%5D%5B%5D=hitbtc&factor%5Bexchanges%5D%5B%5D=hotbit&factor%5Bexchanges%5D%5B%5D=ogre&factor%5Bexchanges%5D%5B%5D=poloniex&factor%5Bexchanges%5D%5B%5D=stex&dataset="

# Build partial query URL
_response = requests.get(url)
_json = _response.json()

# Get dictionary from JSON
coins = _json['coins']

# Import dictionary to dataframe, as the data is flipped we have transposed
_json_df = pd.DataFrame(coins).T

_json_df.head()

,id,tag,algorithm,block_time,block_reward,block_reward24,last_block,difficulty,difficulty24,nethash,...,exchange_rate_curr,market_cap,estimated_rewards,estimated_rewards24,btc_revenue,btc_revenue24,profitability,profitability24,lagging,timestamp
Nicehash-Ethash,15,NICEHASH,Ethash,1,1,1,0,1,1,48377654489224,...,BTC,$0,0.00007,0.00007,0.00006551,0.00006563,101,105,False,1655387467
Ethereum,151,ETH,Ethash,14.8335,2.137787,2.088077,14973639,13532182336868400.0,13605658778179468.0,912271705050621,...,BTC,"$134,673,644,423",0.00123,0.00119,0.0000646,0.00006276,100,100,False,1655387477
Firo,175,FIRO,FiroPow,283.0,6.25,6.25,486261,1837.2669,5634.971738,27883396641,...,BTC,"$19,050,143",2.82356,0.92158,0.00019087,0.0000623,295,99,False,1655387430
Ravencoin,234,RVN,KawPow,60.0,2500.0,2500.0,2323048,34577.745431,34621.307103,2475171429934,...,BTC,"$212,452,011",56.72245,56.65108,0.00005445,0.00005439,84,87,False,1655387295
Nicehash-KawPow,45,NICEHASH,KawPow,1,1,1,0,1,1,607235495146,...,BTC,$0,0.00005,0.00005,0.00004865,0.0000491,75,78,False,1655387466


In [86]:
#Get today's bitcoin price to AUD

today = datetime.date.today()
btc_data = pdr.get_data_yahoo(['BTC-AUD'], 
                          today, 
                          today)['Close']

In [87]:
btc_data.iloc[1,0]

30266.673828125

### TRANSFORM: Filter out relevant columns

In [88]:
df_crypto = _json_df[['tag','algorithm','block_time','block_reward','nethash','timestamp','btc_revenue24']]
df_crypto['AUD_revenue24'] = df_crypto['btc_revenue24'].astype('float').multiply(btc_data.iloc[1,0])

df_crypto.head()

C:\Users\edber\AppData\Local\Temp/ipykernel_20420/448243508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crypto['AUD_revenue24'] = df_crypto['btc_revenue24'].astype('float').multiply(btc_data.iloc[1,0])


,tag,algorithm,block_time,block_reward,nethash,timestamp,btc_revenue24,AUD_revenue24
Nicehash-Ethash,NICEHASH,Ethash,1,1,48377654489224,1655387467,0.00006563,1.986402
Ethereum,ETH,Ethash,14.8335,2.137787,912271705050621,1655387477,0.00006276,1.899536
Firo,FIRO,FiroPow,283.0,6.25,27883396641,1655387430,0.0000623,1.885614
Ravencoin,RVN,KawPow,60.0,2500.0,2475171429934,1655387295,0.00005439,1.646204
Nicehash-KawPow,NICEHASH,KawPow,1,1,607235495146,1655387466,0.0000491,1.486094


### EXTRACT GPU HASRATES + POWER + ALGORITHM

In [89]:
#Scrape usign request to get html of page
page = requests.get('https://whattomine.com/gpus')
b = page.content

# Using Pandas to convert html into dataframe
df_list = pd.read_html(b)
df = df_list[-1]

# Get only the relevant columns
df2 = df[['Model','Hashrate']]

df2.head()


,Model,Hashrate
0,NVIDIA GeForce RTX 3090 Ti (*),128.00 Mh/s Ethash · 350W
1,NVIDIA GeForce RTX 3090,119.00 Mh/s Ethash · 320W
2,NVIDIA GeForce RTX 3080 Ti,117.00 Mh/s Ethash · 330W
3,NVIDIA RTX A5000 (*),99.00 Mh/s Ethash · 230W
4,AMD Radeon VII,93.00 Mh/s Ethash · 200W


### TRANSFORM: Split strings and Append dataframe

In [90]:
# Split Model columns to 'Make' & 'Series'
df12 = df2['Model'].str.split(" ", n=1,expand=True)
# Rename COlumn name 
df12 = df12.rename(columns={0:'GPU_make',1:'GPU_name'})

df12.head()

,GPU_make,GPU_name
0,NVIDIA,GeForce RTX 3090 Ti (*)
1,NVIDIA,GeForce RTX 3090
2,NVIDIA,GeForce RTX 3080 Ti
3,NVIDIA,RTX A5000 (*)
4,AMD,Radeon VII


In [91]:
#Seperate 'Hasrate' Column to three different columns [hashrate, algorithm, power]
df13 = df2['Hashrate'].str.split(" ", n =3, expand=True)

# Removed unused columns 
#df123 = df123.drop(df123.columns[3],axis = 1)

# Rename index columns
df123 = df13.rename(columns={0:'hashrate',1:'units',2:'algorithm',3:'power'})
df123 = df123.astype({'hashrate': float})

# Change all units to MH/s where we have h/s
df123.loc[df123['units'] == 'h/s', 'hashrate'] /= 1000000
# Drop 'units' columns
df123 = df123.drop(df123.columns[1],axis = 1)
# Re-instate dataframe
df123 = df123

# Drop the '.' and 'W' and leave the number
df123['power'] = df123['power'].str.replace('[·W]',"")

df123.head()


C:\Users\edber\AppData\Local\Temp/ipykernel_20420/3818879365.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df123['power'] = df123['power'].str.replace('[·W]',"")


,hashrate,algorithm,power
0,128.0,Ethash,350
1,119.0,Ethash,320
2,117.0,Ethash,330
3,99.0,Ethash,230
4,93.0,Ethash,200


In [92]:
# Combining back dataframe together
df_gpu = df12
df_gpu['hashrate'] =  df123['hashrate']
df_gpu['algorithm'] =  df123['algorithm']
df_gpu['power_consumption'] =  df123['power']

df_gpu.head()

,GPU_make,GPU_name,hashrate,algorithm,power_consumption
0,NVIDIA,GeForce RTX 3090 Ti (*),128.0,Ethash,350
1,NVIDIA,GeForce RTX 3090,119.0,Ethash,320
2,NVIDIA,GeForce RTX 3080 Ti,117.0,Ethash,330
3,NVIDIA,RTX A5000 (*),99.0,Ethash,230
4,AMD,Radeon VII,93.0,Ethash,200


### EXTRACT: COST table from a website 
#### Refer to 'Get_CPUcost.py' for Extract + Transformd

In [93]:
df_cost = Get_GPUcost.get_GPUCost()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\edber\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [94]:
df_cost.head()

,GPU_name,GPU_cost,GPU_make
0,USB 3.0 to HDMI and DVI/VGA Dual Output Extern...,139,ALOGIC
1,Radeon RX 6600 XT Challenger D 8GB GDDR6,639,ASRock
2,Radeon RX 6800 XT Phantom Gaming D OC 16GB GDDR6,1634,ASRock
3,GeForce GT 730 2GB GDDR5,109,ASUS
4,GeForce GT 730 2GB GDDR5,149,ASUS


### Create database connection

In [95]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.CryptoMining_db

# Declare the collection & Insert
db.cost.insert_many(df_cost.to_dict('records'))
db.gpu.insert_many(df_gpu.to_dict('records'))
db.crypto.insert_many(df_crypto.to_dict('records'))


In [96]:
# Verify results:
print(db.cost.count())
print(db.gpu.count())
print(db.crypto.count())


267
57
56


C:\Users\edber\AppData\Local\Temp/ipykernel_20420/2717231059.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print(db.cost.count())
C:\Users\edber\AppData\Local\Temp/ipykernel_20420/2717231059.py:3: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print(db.gpu.count())
C:\Users\edber\AppData\Local\Temp/ipykernel_20420/2717231059.py:4: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere mu